In [1]:
import logging
from six import iteritems
from smart_open import smart_open
import gensim
from gensim import corpora, models, similarities
from gensim.models import TfidfModel
import gensim.downloader as api
from multiprocessing import cpu_count
from gensim.similarities import MatrixSimilarity, WmdSimilarity, SoftCosineSimilarity
import numpy as np
from nltk.corpus import stopwords
from nltk import download

C:\Dev\Anaconda3\envs\gensim_env\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [28]:
documents = []
dictionary = None
tfidf = None
corpus_tfidf = None
corpus = None

def load_documents():
    documents = list(line.lower() for line in smart_open('booksummaries_cleaned.txt', 'rb'))
    return documents

In [3]:
# method to remove common repetitive english words (is, he, she, the, in etc)
def get_stopwords():
    # Download stopwords list.
    download('stopwords')  
    # Remove stopwords.
    stop_words = stopwords.words('english')
    return stop_words

In [4]:
def build_dict():
    stop_words = get_stopwords()
    dictionary = corpora.Dictionary(gensim.utils.simple_preprocess(line) for line in documents)
    # remove stop words and words that appear only once
    stop_ids = [dictionary.token2id[stopword] for stopword in stop_words 
                if stopword in dictionary.token2id]
    
    # since we deal with book titles I kept the one words in our dict
    once_ids = [] #[tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

    # remove stop words and words that appear only once
    dictionary.filter_tokens(stop_ids + once_ids)
    dictionary.save('./dictionary')
    return dictionary
def load_dict():
    return corpora.Dictionary.load('./dictionary')

In [5]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
            for line in open(self.filename, encoding="iso-8859-1"):
                yield gensim.utils.simple_preprocess(line)
                
def read_sentences(fname):
    with smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
                yield gensim.utils.simple_preprocess(line)

In [29]:
def build_corpus():
    corpus = list(dictionary.doc2bow(gensim.utils.simple_preprocess(document)) for document in documents)
    return corpus

In [7]:
def build_tfidf(corpus, dictionary):
    t = TfidfModel(corpus, dictionary=dictionary)
    t.save('./tfidf')
    return t
def load_tfidf():
    return TfidfModel.load('./tfidf')

In [67]:
#tfidf = build_tfidf(corpus, dictionary)

<h1>Load Existing Metadata</h1>

In [30]:
documents = load_documents()
len(documents)

16559

In [31]:
sentences = MySentences('booksummaries_cleaned.txt')

In [32]:
dictionary = load_dict() 

In [33]:
# build_corpus()
# build_tfidf()

In [34]:
#corpus = build_corpus()
tfidf = load_tfidf()

<h2>Corpus tfidf</h2>

In [15]:
corpus_tfidf = tfidf[corpus]

TypeError: 'NoneType' object is not iterable

<h1> Cosin Similarity </h1>

In [81]:
def cossim():
    index = MatrixSimilarity(
        corpus_tfidf,
        num_features=len(dictionary))
    index.save('cossim_index')
    return index

In [17]:
cossim_index = MatrixSimilarity.load('cossim_index') #cossim()

<h1> Model evaluation </h1>

In [35]:
query = "Animal Farm"
query_vec = tfidf[dictionary.doc2bow(query.lower().split())]

In [37]:
# cosin sim
sim = cossim_index[query_vec]

In [38]:
maxSimilarIndx = np.argmax(sim)
sim[maxSimilarIndx]
print("maxSimilarIndx : " + str(maxSimilarIndx) + " similarties_sc[maxSimilarIndx]:" + str(sim[maxSimilarIndx]))

maxSimilarIndx : 9989 similarties_sc[maxSimilarIndx]:0.4097048


In [39]:
sorted_sim = sorted(enumerate(sim), key=lambda item: -item[1])

In [43]:
counter = 1
for idx, val in sorted_sim[:5]:
    print(" #" + str(counter) + " - doc# "+  str(idx) + " - (" + str(val * 100) + ") - Doc:")
    print(" " + str(documents[idx]))
    print(" ==================================================== ")
    counter += 1


 #1 - doc# 9989 - (40.97048044204712) - Doc:
 b'11541759\t3t\tsnowball\'s chance\tjohn reed\t\t{ "parody"}\t the story begins with the death of napoleon, the original antagonist of animal farm. the animals of the farm, fearing what will become of them, learn that snowball is alive and well, and snowball returns to the farm to encourage capitalism. a second windmill is soon built alongside the first, and the two are thenceforth known as the twin mills (allegorical of the twin towers of the world trade center), and the animals all learn to walk on their hind legs, something hitherto forbidden by old major shortly before the expulsion of mr. jones from the farm. also, in place of the original seven commandments, snowball adopts a single slogan for the animals to live by: all animals are born equal - what they become is their own affair. as time passes, the animals, under the leadership of snowball, realise the properties of monetary gain, and begin to file lawsuits against neighbouring fa

<h1> To continue </h1>

    - index file is 7 GB, fix that
    - add more importance to titles / authors